#### Starter template for modeling
Outlines main steps, but may not run all the way through - might need some code to connect sections

In [ ]:
# import libraries
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses

import os
import numpy as np
import pandas as pd
import pickle5 as pickle

import helpers as helper
import seaborn as sns
from sklearn.metrics import confusion_matrix

import time

In [ ]:
# connect Drive if using Colab
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# specify variables for model
path = '/content/drive/MyDrive/W210/Wafer-Defect-Classification'
filename = 'WM-clean-id'
option = '' # -clsaug, -detund
map_column = 'waferMap'
label_column = 'classifyLabels'
filetype = 'zip' # zip, pkl

model_id = 'lenet'
result_path = '/content/drive/MyDrive/W210/Wafer-Defect-Classification'
note = '' # optional

In [ ]:
# load train, dev, and test sets
start = time.time()

if filetype == 'pkl':
    # open pkl files
    with open(f'{path}/{filename}-train{option}.pkl', "rb") as fh:
        train = pickle.load(fh)
    with open(f'{path}/{filename}-dev.pkl', "rb") as fh:
        dev = pickle.load(fh)
    with open(f'{path}/{filename}-test.pkl', "rb") as fh:
        test = pickle.load(fh)

elif filetype == 'zip':
    train = helper.load(f'{path}/{filename}-train{option}.zip')
    dev = helper.load(f'{path}/{filename}-dev.zip')
    test = helper.load(f'{path}/{filename}-test.zip')

print("Wall time: {} seconds".format(time.time() - start))
print(f"Train: {len(train)}")
print(f"Dev: {len(dev)}")
print(f"Test: {len(test)}")

In [ ]:
# for classification models only
# remove none type from dev and test sets
dev = dev[dev.failureType != 'none']
test = test[test.failureType != 'none']

print(f"Dev: {len(dev)}")
print(f"Test: {len(test)}")

In [ ]:
# for classification models using non-augmented data only
# remove none type from train set
train = train[train.failureType != 'none']

print(f"Train: {len(train)}")

#### Quick EDA

In [ ]:
# baseline accuracy of test set
nones = len(test[test.failureType == 'none'])
total = len(test)
print(f"Baseline accuracy: {nones/total*100:.2f}%")

In [ ]:
# train failure type distribution
helper.defect_distribution(train, note='Train Set')

In [ ]:
# dev failure type distribution
helper.defect_distribution(dev, note='Dev Set')

In [ ]:
# test failure type distribution
helper.defect_distribution(test, note='Test Set')

#### Data set-up

In [ ]:
# prepare inputs
x_train = np.stack(train[map_column])
x_val = np.stack(dev[map_column])
x_test = np.stack(test[map_column])

# sanity check
# expected: (#rows, xdim, ydim)
x_train.shape

In [ ]:
# expand tensor and create dummy dimension at axis 3
# images in greyscale, so no channel dimension
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_val = tf.expand_dims(x_val, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)

# sanity check
# expected: TensorShape([#rows, xdim, ydim, 1])
x_train.shape

In [ ]:
# prepare labels for supervised learning
# note: make sure labels are integers if using sparse categorical cross entropy
y_train = np.asarray(train[label_column]).astype('int64')
y_val = np.asarray(dev[label_column]).astype('int64')
y_test = np.asarray(test[label_column]).astype('int64')

# sanity check
type(y_train[0])

In [ ]:
# for classification models using sparse categorical cross entropy only
# adjust labels so they are 0-7 instead of 1-8
# subtract 1 from labels to agree with model
y_train = y_train - 1
y_val = y_val - 1
y_test = y_test - 1

# sanity check
print(min(y_train))
print(max(y_train))

#### Model

In [ ]:
# define model architecture
model = models.Sequential()
model.add(layers.Conv2D(6, 5, activation='tanh', input_shape=x_train.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(120, 5, activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(9, activation='softmax'))
model.summary()

In [ ]:
# set model optimizer and metrics
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
# run model
history = model.fit(x_train, y_train, batch_size=64, epochs=40, validation_data=(x_val, y_val))

In [ ]:
# visualize accuracy and loss history
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

#### Model results

In [ ]:
# compute model results on test set
start = time.time()
results = model.evaluate(x_test, y_test)

print("Wall time: {} seconds".format(time.time() - start))
print(results)

In [ ]:
# generate predictions for model analysis
start = time.time()
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
print("Wall time: {} seconds".format(time.time() - start))

# if googlenet
# pred = model.predict(x_test)
# y_pred = np.argmax(pred[0], axis=1)
# print("Wall time: {} seconds".format(time.time() - start))

In [ ]:
# save results, history, and predictions
with open(f'{results_path}/{model_id}-{filename}{option}{note}.pkl', "wb") as f:
    pickle.dump(y_pred, f)

In [ ]:
# plot confusion matrix
helper.plot_confusion_matrix(y_test, y_pred, normalize=True)